In [1]:
[a for a in [1,2,3] if a not in {'OX'}]

[1, 2, 3]

In [16]:
import torch as th
import itertools
import atomium
import pandas as pd
import params

import calc

SyntaxError: invalid syntax (calc.py, line 2)

In [3]:
data = atomium.fetch('2lyz')
data = data.model

In [4]:
aa_atoms = ['N','C','O','CA','CB','CG','CD','CE','CZ','OD','NH','NE','OG','ND','SG','OE','CH','NZ','OH','SD','OX']

AT_INT = {k : i for i,k in enumerate(aa_atoms)}
INT_AT = {v:k for k, v in AT_INT.items()}

at_at_list = [[f'{at1}-{at2}' for at1 in aa_atoms] for at2 in aa_atoms]
AT_AT_name = list(itertools.chain(*at_at_list))
ATAT_INT = {k : i for i,k in enumerate(AT_AT_name)}
num_atat_feats = len(ATAT_INT)


In [5]:
atoms = []
name = []
CA_xyz = []
residues = []
i = 0
for chain in data.chains():
    for res in chain.residues():
        for atom in res.atoms():
            n = atom.name
            if n == 'CA':
                CA_xyz.append(atom.location)
            elif len(n) == 3:
                n = n[:2]
            name.append(n)
            atoms.append(atom.location)
            residues.append(i)
        i += 1

In [6]:
AT_xyz = th.FloatTensor(atoms)
AT_dist = th.cdist(AT_xyz, AT_xyz)

In [7]:
num_resid = len(CA_xyz)
xyz = th.FloatTensor(atoms)
dist = th.cdist(xyz, xyz)
res_num = th.LongTensor(residues)
atom_id = [AT_INT[at] for at in name]
atom_id = th.LongTensor(atom_id)

In [8]:
dist_where = (dist < 2)
res1, res2 = th.nonzero(dist_where, as_tuple=True)

In [9]:
aa1, aa2 = res_num[res1], res_num[res2]
at1, at2 = atom_id[res1], atom_id[res2]
inter = aa1 != aa2
inter1, inter2 = aa1[inter], aa2[inter]

In [10]:
print('num:', inter.shape)
bond_list = list()
bond_dict = {i : [] for i in range(num_resid)}
for i in range(inter1.shape[0]-1):
    rid1, rid2 = inter1[i].item(), inter2[i].item()
    atid1, atid2 = at1[i].item(), at2[i].item()
    #print(rid1, ' - ', rid2, ': ', INT_AT[atid1], '-',  INT_AT[atid2])
    atat_name = INT_AT[atid1]+'-'+INT_AT[atid2]
    bond_list.append(atat_name)
    bond_dict[rid1].append(atat_name)

num: torch.Size([3059])


In [11]:


def bond_to_vector(bond_dict):
    stack = list()
    for res, bond_list in bond_dict.items():
        
        bonds_unique = set(bond_list)
        bonds = th.zeros(params.num_atat_feats)
        
        if bonds_unique:
            
            bonds_id = [params.ATAT_INT[b] for b in bonds_unique if b in params.ATAT_INT]
            bonds[bonds_id] = 1
        stack.append(bonds.unsqueeze(0))
    feats = th.cat(stack, 0)
    return feats